### Note:
This notebook submit jobs to the clusters. For each simulation, the algorithm will calculate the geometric features. These features were then further used for migratory modes classification.

In order to use this notebook, some of the paths might need to be adapted.

In [1]:
import numpy as np
import sys
from datetime import datetime
import subprocess
import os

import pipe_util2

In [2]:
def parameter_txt(param_folder, script_py_file, 
                  data_folder, feature_folder, figure_folder,
                  repeats, iterations, timepoints
                 ):
    """
    Creating parameter files in designated folders
    that can be later called in sbatch array.

    rep - int, number of repeat for each parameter combinations
    param_folder - str, path to folder that save parameter files
    script_py_file - str, path to the .py file
    data_folder - str, path to the folder with simulation results
    feature_folder - str, path to the folder storing microtumor geometric traits
    figure_folder - str, path to the folder storing simulation snapshot of the analyzed time points

    repeats, iterations - int
    timepoints - list(int), list time points of interest

    return - int, the number of parameter files generated
    """
    param_folder = pipe_util2.folder_verify(param_folder)
    param_folder = pipe_util2.create_folder(param_folder)
    
    count = 0
    i = 0
    while i<repeats:
        
        j = 0
        while j<iterations:
            
            k = 0
            while k<len(timepoints):
                
                file_num = str(count)
                f = open(param_folder+file_num+".p", "w")
                
                line = "python %s %s %s %s %s %s %s"%(
                    script_py_file, 
                    data_folder, feature_folder, figure_folder,
                    i, j, timepoints[k]
                )
                f.write(line)
                f.close()
                
                count = count+1
                k = k+1
            j = j+1
        i = i+1
        
    return repeats*iterations*len(timepoints)

In [3]:
def sbatch_file_polarity_sim(
    param_folder, scr_folder, name = None):
    """
    This function is to generate the sbatch file,
    facilitating submitting geometry feature analysis jobs to the cluster. 

    param_folder - str, path to folder for storing parameter files.
    scr_folder - str, path to folder for storing both script .sh files and the output .o files.
    name - str, simulation name
    """
    
    now = datetime.now()
    dt_string = now.strftime("%d-%m-%Y-%H%M")
    scr_folder = pipe_util2.create_folder(scr_folder)
    param_folder = pipe_util2.folder_verify(param_folder)
    
    if name:
        ofile_name = scr_folder + dt_string + "geo_calc_" + name + ".o"
        sfile_name = scr_folder + dt_string + "geo_calc_" + name + ".sh"
        
    else:
        ofile_name = scr_folder + dt_string + "geo_calc" + ".o"
        sfile_name = scr_folder + dt_string + "geo_calc" + ".sh"
    
    ofile_name = os.path.abspath(ofile_name)
    sfile_name = os.path.abspath(sfile_name)
    
    contents = """#!/bin/bash
#SBATCH --account=huijing
#SBATCH --partition=dept_cpu
#SBATCH --cpus-per-task=1
#SBATCH --mem=3G
#SBATCH --output=%s%s_%s.o
#SBATCH --job-name=potts_scan

eval "$(conda shell.bash hook)"
module load anaconda
source activate mg1
echo 'env activated'

bash %s$SLURM_ARRAY_TASK_ID.p
    """%(scr_folder, "%A", "%a", param_folder)
    s = open(sfile_name, "w")
    s.write(contents)
    s.close()
    
    return sfile_name

In [ ]:
script_py_file = "path/to/feature_calc_6_simple_corr_edit.py"
script_py_file = os.path.abspath(script_py_file)

analysis_folder = "path/to/analysis/folder/20250129-msmm-model-analysis"
pipe_util2.create_folder(analysis_folder)
analysis_folder = os.path.abspath(analysis_folder)

data_folder = "path/to/data/folder/Migration/CC3D_data_12222021_force_fpp_scan/"

figure_folder = analysis_folder+"/"+"12222021_force_fpp_scan_calc6_figures/"
feature_folder = analysis_folder+"/"+"12222021_force_fpp_scan_calc6_features/"
pipe_util2.create_folder(figure_folder)
pipe_util2.create_folder(feature_folder)

param_folder = analysis_folder+"/"+"12222021_force_fpp_scan_param/"
scr_folder = analysis_folder+"/"+"12222021_force_fpp_scan_script/"

repeats = 50
iterations = 231
timepoints = [90,270]
jobmax_num = parameter_txt(param_folder, script_py_file, 
                           data_folder, feature_folder, figure_folder,
                           repeats, iterations, timepoints
                          )

sbatch_file = sbatch_file_polarity_sim(
    param_folder, scr_folder, name = None)

batch_size = 500
if jobmax_num<batch_size:
    batch_size = jobmax_num+1
batch_size = "%"+"%s"%batch_size
cmd = "sbatch --array=[0-%s]%s %s"%(jobmax_num, batch_size, sbatch_file)
subprocess.call(cmd, shell=True)
cmd